# Setup

In [2]:
from __future__ import annotations
import functools
from pathlib import Path
from typing import Any

import pandas as pd
import plotly.express as px

## Config

In [13]:
INSTANCE_DIR = Path('../../../instances')
TIMEOUT_SECS = 60**2

## Pre-processing results

In [13]:
for results_dir in Path('results').glob('*'):
    combined_file = results_dir / 'combined.csv'
    csv_files = list(results_dir.glob('**/*.csv'))
    try:
        csv_files.remove(combined_file)
    except ValueError:
        pass
    with combined_file.open('w') as f:
        for i, csv_file in enumerate(csv_files):
            text = csv_file.read_text()
            if i > 0:
                text = text.split('\n', maxsplit=1)[1]
            f.write(text)

## Loading results

In [4]:
all_instance_names = [
    f.name for f in INSTANCE_DIR.glob('**/*.dat')
    if f.relative_to(INSTANCE_DIR).parts[0] != 'outdated'
]

def load_combined_df(spec: dict[str, str]) -> pd.DataFrame:
    dfs = (pd.read_csv(f'results/{dir_name}/combined.csv',
                       index_col='file_name').add_suffix('_' + suffix)
           for suffix, dir_name in spec.items())
    combined_df = functools.reduce(lambda df1, df2: df1.join(df2), dfs)

    # Make sure all datasets have one row per instance, even if the instances
    # didn't finish running in any experiment (and thus has no row in any CSV)
    return combined_df.reindex(all_instance_names)


# How many vertices to check in the static packing costly discard rule?